In [1]:
## script to generate Scispot registrations for ChESS 96-well plate
## lkp 2022/11/15

import pandas as pd
import requests
import json
import sys

sys.stdout.write("Imported required packages")

Imported required packages

# Jupyter notebook in Scispot

- Scispot --> LabSheets --> Jupyter --> +New --> write script/notebook
- - need jupyterhub server credentials?

- ? can I look up a UUID by sample name?
- - update-by-id to fetch UUID from Sample ID
- ? can I look up UUIDs by manifest name?
- - not yet! 
- ? can I specify other "variable" inputs/params? or can scientist edit the ipynb with those values?
- - A: yes, can code input parameters 
- - ? can I read in other sheets into pandas?
- - - nm don't worry about this for now...


### tips
- update-by-id to fetch UUID from Sample ID 
- https://www.loom.com/share/32c01c2cb1294720b15596b0a43af5e9

In [118]:
# lookup an entry by UUID
def find_entry_from_uuid(manager, uuid_list):
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/find-row"
    payload = {
        "apiKey": "3F4E0776-EFEF-4A5B-8479-01B7C2367090",
        "manager": manager,
        "uuid": uuid_list
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

test_uuid = "0344a99d-888a-443e-9a23-13eceb279ae1"
find_entry_from_uuid('Sample Manager', test_uuid)

{'headers': ['UUID',
  'Sample ID',
  'Working compound',
  'Working cell line',
  'Pert Dose',
  'Pert Time',
  'Pert Protocol',
  'Tissue ID',
  'Tissue Passage',
  'Type',
  'Parent',
  'Children',
  'Location',
  'Storage Form',
  'Storage Amount',
  'Pass',
  'Quantity Number of Cells (x1000)',
  'Created By'],
 'row': ['0344a99d-888a-443e-9a23-13eceb279ae1',
  'THP1_dbgen1_VTP50469_3',
  'VTP50469_1',
  'THP1_dbgen1',
  '',
  '',
  '',
  '',
  '',
  'Frozen Nuclei',
  '90b19820-726d-4086-b74a-199d37899ea5',
  '',
  'ChESS_dbgen1_202212_plate1rep3',
  '',
  '',
  '',
  '0.000',
  ''],
 'success': 'true'}

In [82]:
# turn json into pandas df
def json_to_pd_df(json_in):
    rows = json_in["row"]
    column_labels = json_in["headers"]
    df_out = pd.DataFrame(data=rows, columns=column_labels)
    return df_out

temp = find_entry_from_uuid(test_uuid)
temp_df = pd.DataFrame(data=find_entry_from_uuid(test_uuid)['row']).T
temp_df.columns = list(temp['headers'])
temp_df

,UUID,Sample ID,Working compound,Working cell line,Pert Dose,Pert Time,Pert Protocol,Tissue ID,Tissue Passage,Type,Parent,Children,Location,Storage Form,Storage Amount,Pass,Quantity Number of Cells (x1000),Created By
0,0344a99d-888a-443e-9a23-13eceb279ae1,THP1_dbgen1_VTP50469_3,VTP50469_1,THP1_dbgen1,,,,,,Frozen Nuclei,90b19820-726d-4086-b74a-199d37899ea5,,ChESS_dbgen1_202212_plate1rep3,,,,0.000,


In [100]:
# fetch any row from the Protein Manager to get the columns
def fetch_entry():
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/list-rows"
    payload = {
        "apiKey": "3F4E0776-EFEF-4A5B-8479-01B7C2367090",
        "manager": "Protein Manager",
        "pageSize": "1",
        "page": "1"
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

# initialize an empty Protein Manager
prot_df = pd.DataFrame(data=fetch_entry()['rows'], columns=fetch_entry()['headers'])
prot_df= prot_df.drop(prot_df.index[0])
prot_df

,UUID,Protein ID,Sample ID,Protocol ID,Extraction Date,Extraction Scientist,Extraction Fraction,Extraction Notes,Protein Concentration (ug/mL),BCA Results,Location,Storage Form,Storage Amount (uL),Pass,Parent,Children,Created By,Creation Date


In [113]:
import datetime

## sample specific variables
# name the Protein ID after the Sample ID
prot_df['Sample ID'] = temp_df['Sample ID']
prot_df['Protein ID'] = temp_df['Sample ID'] + "_chromatin"

# link the Parent Sample ID
prot_df['Parent'] = temp_df['UUID']

## batch specific variables
extraction_date = "12/01/2022"
extraction_scientist = "Lindsay"
extraction_fraction = "chromatin"
created_by = "Lindsay"
creation_date = datetime.datetime.now().strftime("%x")
prot_df['Extraction Date'] = extraction_date
prot_df['Extraction Scientist'] = extraction_scientist
prot_df['Extraction Fraction'] = extraction_fraction
prot_df['Created By'] = created_by
prot_df['Creation Date'] = creation_date


# extraction scientist

list(prot_df.iloc[0][1:])

['THP1_dbgen1_VTP50469_3_chromatin',
 'THP1_dbgen1_VTP50469_3',
 nan,
 '12/01/2022',
 'Lindsay',
 'chromatin',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '0344a99d-888a-443e-9a23-13eceb279ae1',
 nan,
 'Lindsay',
 '03/02/23']

In [126]:
# add a new row to the Protein Manager
def add_entry1(new_row):
    print(new_row)
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/add-rows"
    payload = {
        "apiKey": "3F4E0776-EFEF-4A5B-8479-01B7C2367090",
        "manager": "Protein Manager",
        "rows": [new_row]
    }
    ret = session.post(url, json=payload)
    print(response.text)

def add_entry(new_row):
    url = "https://api.scispot.io/tryingtofixcors/labsheets/add-rows"
    payload = {
        "apiKey": "3F4E0776-EFEF-4A5B-8479-01B7C2367090",
        "manager": "Protein Manager",
        "rows": [new_row]
    }
    headers = {"Content-Type": "application/json"}
    response = requests.request("POST", url, json=payload, headers=headers)
    print(response.text)
    
new_row = list(prot_df.iloc[0][:1])
add_entry(new_row)

{"timestamp":"2023-03-03T00:26:18.968+00:00","status":400,"error":"Bad Request","message":"","path":"/labsheets/add-rows"}


In [119]:
## TODO check if row added? THP1_dbgen1_VTP50469_3
new_uuid = "0344a99d-888a-443e-9a23-13eceb279ae1"
find_entry_from_uuid('Protein Manager', new_uuid)


{'headers': ['UUID',
  'Protein ID',
  'Sample ID',
  'Protocol ID',
  'Extraction Date',
  'Extraction Scientist',
  'Extraction Fraction',
  'Extraction Notes',
  'Protein Concentration (ug/mL)',
  'BCA Results',
  'Location',
  'Storage Form',
  'Storage Amount (uL)',
  'Pass',
  'Parent',
  'Children',
  'Created By',
  'Creation Date'],
 'row': ['0344a99d-888a-443e-9a23-13eceb279ae1',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'ChESS_dbgen1_202212_plate1rep3',
  '',
  '',
  '',
  '90b19820-726d-4086-b74a-199d37899ea5',
  '',
  '',
  ''],
 'success': 'true'}

In [120]:
find_entry_from_uuid('Sample Manager', '90b19820-726d-4086-b74a-199d37899ea5')

{'headers': ['UUID',
  'Sample ID',
  'Working compound',
  'Working cell line',
  'Pert Dose',
  'Pert Time',
  'Pert Protocol',
  'Tissue ID',
  'Tissue Passage',
  'Type',
  'Parent',
  'Children',
  'Location',
  'Storage Form',
  'Storage Amount',
  'Pass',
  'Quantity Number of Cells (x1000)',
  'Created By'],
 'row': ['90b19820-726d-4086-b74a-199d37899ea5',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '98f8e123-84ab-4171-b26e-6c3c0ad6ecf3, 83ec0c18-c91c-429e-9e98-c983c8870230, 05e5854e-3c57-4290-a6c5-0a125f24bb20, ac7003a6-5ec1-443d-9487-d08f19216adc, 2d4293e1-5655-4b70-9cb0-7001d8480375, c55f6eb7-dfbb-4b3b-b737-580eb8ea9ebc, b0643361-88c7-414a-9e03-33b896369026, f837648a-e2ca-4e63-8d15-f269faa553a1, bf6d3d5b-3853-4769-8149-7f08fe791eeb, 12d5bcba-5cd4-4614-a192-1c8441b9c04c, c83f7225-3c8f-4098-a833-80cd77ee9fb8, 65ead75a-1064-4c5e-b3bc-8544bba2557c, 8a4dc3e8-b7c8-48c1-8cda-ae63f499a476, 95979a83-1389-49a8-ba37-fca9901b5afb, 27f8028a-40e7-4512-8676-6598dff907d8, 

In [121]:
find_entry_from_uuid('Sample Manager', '98f8e123-84ab-4171-b26e-6c3c0ad6ecf3')

{'headers': ['UUID',
  'Sample ID',
  'Working compound',
  'Working cell line',
  'Pert Dose',
  'Pert Time',
  'Pert Protocol',
  'Tissue ID',
  'Tissue Passage',
  'Type',
  'Parent',
  'Children',
  'Location',
  'Storage Form',
  'Storage Amount',
  'Pass',
  'Quantity Number of Cells (x1000)',
  'Created By'],
 'row': ['98f8e123-84ab-4171-b26e-6c3c0ad6ecf3',
  'THP1_dbgen1_DMSO_1',
  'DMSO-1',
  'THP1_dbgen1',
  '',
  '',
  '',
  '',
  '',
  '',
  '90b19820-726d-4086-b74a-199d37899ea5',
  'b121392b-68b9-4557-b3b3-8210132bf8d3, 18418a4d-236b-4c08-b595-4a3499e72f76, fb33830a-ad4b-4eee-8364-01c513c4cabd, 1891eeac-62c4-44f1-94e7-647a02ce3b17, aa18c698-f22c-4814-9c08-45f8b7452f1e',
  'ChESS_dbgen1_plate1',
  '',
  '',
  '',
  '0',
  ''],
 'success': 'true'}

In [ ]:
# someday I should actually make objects for scispot stuff...
# TODO: read in headers from API? so that if they change, this isn't all hardcoded...
class scispot_managers:
    def __init__( self, data, ):
        self.sample = pd.DataFrame()
        self.protein = pd.DataFrame()
        self.peptide = pd.DataFrame()
        
    def sample_to_protein( self, input ):
        self.peptide[ 'new_column' ] = input

def initiate_sample_df():
    df = pd.DataFrame(columns=['Sample ID',
                                 'Working compound',
                                 'Working cell line',
                                 'Pert Dose',
                                 'Pert Time',
                                 'Pert Protocol',
                                 'Tissue ID',
                                 'Tissue Passage',
                                 'Type',
                                 'Storage Form',
                                 'Storage Amount',
                                 'Pass',
                                 'Quantity Number of Cells (x1000)',
                                 'Created By'])
    return(df)

def initiate_protein_df(sample_df, frx):
    df = pd.DataFrame(columns=['Protein ID',
                                 'Sample ID',
                                 'Protocol ID',
                                 'Extraction Date',
                                 'Extraction Scientist',
                                 'Extraction Fraction',
                                 'Extraction Notes',
                                 'Protein Concentration (ug/mL)',
                                 'BCA Results',
                                 'Location',
                                 'Storage Form',
                                 'Storage Amount (uL)',
                                 'Pass',
                                 'Created By',
                                 'Creation Date'])
    
    df['Sample ID'] = sample_df['Sample ID']
    df['Protein ID'] = sample_df['Sample ID'].astype(str) + '_' + frx
    df['Extraction Fraction'] = frx
    # TODO add datetime ['Creation Date']
    # TODO add log-in ['Created By']
    # TODO inherit Sample ID UUID from Sample ID 
    # TODO add 
    
    return(df)

def initiate_peptide_df(input_df):
    df = pd.DataFrame(columns=['Digestion ID',
                                 'Protein ID',
                                 'Digestion Date',
                                 'Protocol ID',
                                 'Digestion Scientist',
                                 'Protein Input',
                                 'Reduction Chemical',
                                 'Reduction Dose',
                                 'Alkylation Chemical',
                                 'Alkylation Dose',
                                 'Protease Chemical',
                                 'Protease Dose',
                                 'Quench Chemical',
                                 'Quench Dose',
                                 'Storage Location',
                                 'Storage Form',
                                 'Storage Concentration',
                                 'Pass',
                                 'Location',
                                 'Created By'])
    
    df['Protein ID'] = input_df['Protein ID']
    df['Digestion ID'] = input_df['Protein ID'].astype(str) + '_peptide'
    # add datetime ['Creation Date']
    # add log-in ['Created By']
    
    # populate some default values
    df['Reduction Chemical'] = 'DTT'
    df['Alkylation Chemical'] = 'IAA'
    df['Protease Chemical'] = 'Trypsin'
    
    return(df)

## TODO: what columns to inherit from Sample? Plate position? populate default values?

In [53]:
# lookup entries by Sample ID 
def fetch_uuid(sample_list):
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/list-rows"
    payload = {
        "apiKey": "3F4E0776-EFEF-4A5B-8479-01B7C2367090",
        "manager": "Sample Manager",
        "Sample ID": sample_list
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

sample_list = "THP1_dbgen1_VTP50469_3"
fetch_uuid(sample_list)

{'headers': ['UUID',
  'Sample ID',
  'Working compound',
  'Working cell line',
  'Pert Dose',
  'Pert Time',
  'Pert Protocol',
  'Tissue ID',
  'Tissue Passage',
  'Type',
  'Parent',
  'Children',
  'Location',
  'Storage Form',
  'Storage Amount',
  'Pass',
  'Quantity Number of Cells (x1000)',
  'Created By'],
 'success': 'false',
 'message': 'null'}

In [59]:
# read in a Scispot plate TSV export of Sample Manager
# note: export doesn't include UUID
## TODO possible to pull a plate of samples with some manifest name/location from API?
sample_fi = "G:/My Drive/Lindsay Pino/proj/2023_scispot_utils/data/Sample-2023-03-02T23_27_01.tsv"
input_df = pd.read_table(sample_fi, sep='\t', engine="python")
input_df.head()


,ï»¿Sample ID,Working compound,Working cell line,Pert Dose,Pert Time,Pert Protocol,Tissue ID,Tissue Passage,Type,Storage Form,Storage Amount,Pass,Quantity Number of Cells (x1000),Created By
0,220423_MM1S_Silac_Light_Nuclei-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frozen,NaN,Pass,4000000.0,NaN
1,220423_MM1S_Silac_Light_Nuclei-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frozen,NaN,Pass,4000000.0,NaN
2,220423_MM1S_Silac_Light_Nuclei-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frozen,NaN,Pass,4000000.0,NaN
3,220423_MM1S_Silac_Light_Nuclei-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frozen,NaN,Pass,4000000.0,NaN
4,220423_MM1S_Silac_Heavy_Nuclei-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frozen,NaN,Pass,4000000.0,NaN


In [ ]:
## GENERATE PROTEIN SAMPLES
## TODO: this is pretty inelegant, could perhaps reduce this to a list of fractions to generate?

# generate protein samples: cytosol
protein_df_cyto = initiate_protein_df(input_df, 'Cytosolic')

# generate protein samples: nucleoplasm
protein_df_nucleo = initiate_protein_df(input_df, 'Nucleoplasm')

# generate protein samples: euchromatin
protein_df_euch = initiate_protein_df(input_df, 'Euchromatin')

# generate protein samples: heterochromatin
protein_df_het = initiate_protein_df(input_df, 'Heterochromatin')

# generate protein samples: insoluble
protein_df_ins = initiate_protein_df(input_df, 'Insoluble')

# append all protein to one protein dataframe
protein_df = protein_df_cyto.append(protein_df_nucleo.append(protein_df_euch.append(protein_df_het.append(protein_df_ins))))

# 


In [ ]:

print(protein_df.head())
print(peptide_df.head())